In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint

In [2]:
df = pd.read_csv("data/baseline_cases_runs.csv")
reference = "nawi"

In [3]:
for source_water in ["agriculture"]: #df.source_water_category.unique():
    for case_study in df[df.source_water_category == source_water].case_study.unique():
        h1 = df[df.case_study == case_study]

        if case_study in ["upw"]:
            scenarios = ["zld_a", "zld_b", "zld_c", "baseline"]
        else:        
            scenarios = h1.scenario.unique()

        for scenario in scenarios:

            print("------------------------------------------------------")
            print("------------------------------------------------------")
            print("-----------", case_study, scenario, "-----------------")
            print("------------------------------------------------------")
            print("------------------------------------------------------")

            desired_recovery = df[((df.case_study == case_study) & (df.scenario == scenario))].max_recovery_rate.max()

            source_water_category = df[((df.case_study == case_study) & (df.scenario == scenario))].source_water_category.max()

            skip_small_base = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_base.max()
            skip_small_sens = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_sens.max()
            ro_bounds = df[((df.case_study == case_study) & (df.scenario == scenario))].ro_bounds.max()

            m = wt.watertap_setup(dynamic=False, case_study = case_study, reference = reference, 
                                  scenario = scenario, source_scenario = "baseline")

            m = wt.case_study_trains.get_case_study(m=m)

            for key in m.fs.pfd_dict.keys():
                if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
                    getattr(m.fs, key).a.fix(4.2)
                    getattr(m.fs, key).b.fix(0.35)

            if "1p5" in scenario:
                m.fs.irrigation_and_drainage.flow_vol_in.fix(0.0657)
            
            ############################################################
            ############################################################
            # set recovery for uranium case RO units.
            #if case_study == "uranium":
                
            
            
            
            
            # run and and return baseline with 0 degrees of freedom
            m = wt.run_water_tap_ro(m, source_water_category = source_water_category, 
                                 desired_recovery = desired_recovery, skip_small = skip_small_base, 
                                 ro_bounds = ro_bounds, source_scenario = "baseline", scenario_name = scenario)

            if scenario == "baseline":
                wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)

                if source_water == "seawater":
                    m = wt.get_fixed_onm_reduction(m)
                    wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                    df_no2 = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], 
                                              scenario="50%_fixed_onm_reduction")

                if case_study == "san_luis":
                    n = 1
                    for flow_scenario in [0.0657, 0.1314, 0.2191]:
                        m.fs.irrigation_and_drainage.flow_vol_in.fix(flow_scenario)
                        m.fs.reverse_osmosis.feed.pressure.unfix()
                        m.fs.reverse_osmosis.membrane_area.unfix()

                        print("adjusted baseline flow -->", flow_scenario)
                        wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                        if n == 1: s_name = "1p5_mgd"
                        if n == 2: s_name = "3_mgd"
                        if n == 3: s_name = "5_mgd"
                        df_no2 = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)                
                        n = n + 1
                        
            if case_study == "lithium":
                ### double cap
                s_name = scenario + "x2 cap"
                flow_in_x2 = m.fs.lithium_brine.flow_vol_in[0]() * 2
                m.fs.lithium_brine.flow_vol_in.fix(flow_in_x2)
                print("adjusted baseline flow -->", flow_in_x2)
                wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                df_no2 = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)
                
            
            
            
            
            dwi_list = ["emwd", "big_spring"]   
            if m.fs.train["case_study"] in dwi_list:
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "deep_well_injection":
                        wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)

            if scenario in ["edr_ph_ro", "ro_and_mf"]:
                print("goes in")
                wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)                

            if source_water == "power":
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "evaporation_pond":
                        wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)  



------------------------------------------------------
------------------------------------------------------
----------- monterey_one baseline -----------------
------------------------------------------------------
------------------------------------------------------
monterey_one

------- Adding Unit Processes -------
water_pumping_station
coag_and_floc
dual_media_filtration
chlorination
treated_storage
intrusion_mitigation
landfill
-------------------------------------

adding source

Connecting unit processes...
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WATER PUMPING STATION:

	total cap investment: 

Plant Capacity Utilization 70-100% 1.0000000000000002 LCOW --> 0.2765657338986648
------- RESET -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.354571453716237
------- Weighted Average Cost of Capital 5-10% -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.05 LCOW --> 0.354571453716237
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.052500000000000005 LCOW --> 0.36240480117876506
---------------------------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 0.7614000000000001 LCOW --> 0.35457145365435755
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 0.7825500000000001 LCOW --> 0.3545714536698274
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 0.8037000000000001 LCOW --> 0.35457145368529724
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 0.8248500000000002 LCOW --> 0.35457145370076715
-----------------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 1.5115358500000005 LCOW --> 0.3471557193608424
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 1.5443953250000002 LCOW --> 0.3460581671715415
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 1.5772548000000004 LCOW --> 0.34499458265914873
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 1.6101142750000006 LCOW --> 0.3439632167100534
---------------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.10987999999999998 LCOW --> 0.35196115320757665
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.11389999999999997 LCOW --> 0.3523962032923533
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.11791999999999997 LCOW --> 0.3528312533771301
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.12193999999999997 LCOW --> 0.35326630346190685
--------------------

...

adjusted baseline flow --> 0.2191
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


FEED WATER TANK:

	total cap investment: 1.25062
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.01659

DRAINAGE SUMP PUMP1:

	total cap investment: 0.36306
	cat and chem cost: 0.0
	electricity cost: 0.09486
	total fixed op cost: 0.00482

DRAINAGE SUMP PUMP2:

	total cap investment: 0.36303
	cat and chem cost: 0.0
	electricity cost: 0.02371
	total fixed op cost: 0.00482

RAW WATER PUMP1:

	total cap investment: 0.363
	cat and chem cost: 0.0
	electricity cost: 0.02371
	total fixed op cost: 0.00482

RAW WATER PUMP2:

	total cap investment: 0.36297
	cat and chem cost: 0.0
	electricity cost: 0.02371
	total fixed op cost: 0.00482

SODIUM HYPOCHLORITE 1:

	total cap investment: 0.02072
	cat and chem cost: 


WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


FEED WATER TANK:

	total cap investment: 0.16958
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00225

DRAINAGE SUMP PUMP1:

	total cap investment: 0.03055
	cat and chem cost: 0.0
	electricity cost: 0.00546
	total fixed op cost: 0.00041

DRAINAGE SUMP PUMP2:

	total cap investment: 0.03055
	cat and chem cost: 0.0
	electricity cost: 0.00137
	total fixed op cost: 0.00041

RAW WATER PUMP1:

	total cap investment: 0.03055
	cat and chem cost: 0.0
	electricity cost: 0.00137
	total fixed op cost: 0.00041

RAW WATER PUMP2:

	total cap investment: 0.03055
	cat and chem cost: 0.0
	electricity cost: 0.00137
	total fixed op cost: 0.00041

SODIUM HYPOCHLORITE 1:

	total cap investment: 0.00355
	cat and chem cost: 0.00013
	electricity cost: 0.0
	total fixed op cost: 5e-05

MOLASSES ADDITION:

	total cap investment: 0.0096
	cat and chem co


Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Model finished running to initialize conditions. Now running with actual flow...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


FEED WATER TANK:

	total cap investment: 0.16958
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00225

DRAINAGE SUMP PUMP1:

	total cap investment: 0.03055
	cat and chem cost: 0.0
	electricity cost: 0.00546
	total fixed op cost: 0.00041

DRAINAGE SUMP PUMP2:

	total cap investment: 0.03055
	cat and chem cost: 0.0
	electricity cost: 0.00137

glycerin_addition
hydrochloric_acid_addition_1
bioreactor_feed_pump
abmet_interstage_pump
abmet_intermediate_pump
bioreactor
bioreactor_effluent_pump
bioreactor_bw_pump
sodium_hypochlorite_2
nano_feed_pump
nano_filtration
hydrochloric_acid_addition_2
sodium_bisulfite_addition
anti_scalant_addition
reverse_osmosis_a
ph_adjustment
reverse_osmosis_b
agriculture
smp
-------------------------------------

adding source

Connecting unit processes...
Unfixing feed presure and area for reverse_osmosis_a ...

Unfixing feed presure and area for reverse_osmosis_b ...

Flow is relatively small (< 1 m3/s). Running model with larger dummy flows to initialize...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Model finished running to initialize conditions. Now running with actual flow...

------------------------------------------------------------------

san_luis

------- Adding Unit Processes -------
feed_water_tank
drainage_sump_pump1
drainage_sump_pump2
raw_water_pump1
raw_water_pump2
sodium_hypochlorite_1
molasses_addition
glycerin_addition
hydrochloric_acid_addition_1
bioreactor_feed_pump
abmet_interstage_pump
abmet_intermediate_pump
bioreactor
bioreactor_effluent_pump
bioreactor_bw_pump
sodium_hypochlorite_2
uf_feed_pump
ultra_filtration
hydrochloric_acid_addition_2
sodium_bisulfite_addition
anti_scalant_addition
reverse_osmosis
agriculture
deep_well_injection
-------------------------------------

adding source

Connecting unit processes...
Unfixing feed presure and area for reverse_osmosis ...

Flow is relatively small (< 1 m3/s). Running model with larger dummy flows to initialize...

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Model finished running to initialize conditions. No

In [ ]:
----------------------------------------------------------------------
	Feed pressure for reverse_osmosis: 58.83 bar
	Membrane area for reverse_osmosis: 1462.03 m2
	Pure Water Flux for reverse_osmosis: 24.016650044040567 lmh
	A constant for reverse_osmosis: 4.2
	B constant for reverse_osmosis: 0.35
------- Plant Capacity Utilization 70-100% -------

In [8]:
wt.display.show_train2(model_name=m)

In [ ]:
#wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)

In [ ]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))